# Trabalho de Mineração de Dados
### Por: Davi de Castro e João Pedro Righi

* Tentamos requisitar e usar o json diretamente mas havia um erro nele.

* Havia um erro no json onde faltava uma vírgula em determinado item, 
por ser apenas um caso isolado escolhemos mudar manualmente.

* Caso fosse um erro recorrente poderíamos utilizar uma lógica de verificação e inserção de vírgulas faltantes

In [13]:
#  Substituir "\u00e3" por "a"

import json

def substituir_unicode(input_file, output_file):
    # Carrega o conteúdo do arquivo JSON original
    with open(input_file, 'r', encoding='utf-8') as file:
        data = json.load(file)
    
    # Converte o JSON para uma string
    json_str = json.dumps(data)
    
    # Substitui "\u00e3" por "a"
    json_str_modificado = json_str.replace("\u00e3", "a")
    
    # Converte a string de volta para um objeto JSON
    data_modificado = json.loads(json_str_modificado)
    
    # Salva o resultado em um novo arquivo
    with open(output_file, 'w', encoding='utf-8') as file:
        json.dump(data_modificado, file, ensure_ascii=False, indent=4)

# Caminho para os arquivos de entrada e saída
arquivo_entrada = 'padaria_trab.json'
arquivo_saida = 'saida.json'

# Chama a função para substituir o unicode
substituir_unicode(arquivo_entrada, arquivo_saida)

print(f"Substituição concluída. O resultado foi salvo em {arquivo_saida}.")


Substituição concluída. O resultado foi salvo em saida.json.


* Havia um erro de formatação que foi resolvido substituindo "\u00e3" por "a". 

* Agora vamos remover a chave "compra", uma vez que o índice do objeto já será o bastante para diferenciá-lo dos demais.

In [14]:
import json
import unicodedata

def remover_acentos(texto):
    """Remove acentos de um texto."""
    return ''.join(
        c for c in unicodedata.normalize('NFD', texto)
        if unicodedata.category(c) != 'Mn'
    )

def processar_json(input_file, output_file):
    """Lê um JSON, remove a chave 'compra', e remove acentos das vogais."""
    # Carregar os dados do arquivo JSON
    with open(input_file, 'r') as file:
        compras = json.load(file)

    # Processar cada compra
    for compra in compras:
        # Remover a chave 'compra'
        if 'compra' in compra:
            del compra['compra']
        
        # Remover acentos dos produtos
        compra['produtos'] = [remover_acentos(produto) for produto in compra['produtos']]

    # Salvar os dados modificados em um novo arquivo JSON
    with open(output_file, 'w') as file:
        json.dump(compras, file, indent=4)

    print(f"A chave 'compra' foi removida e os acentos foram removidos. Dados salvos em '{output_file}'.")

# Chamar a função com os nomes dos arquivos
processar_json('saida.json', 'nova_saida.json')


A chave 'compra' foi removida e os acentos foram removidos. Dados salvos em 'nova_saida.json'.


* Agora vamos separar os produtos em tipo e categoria e deixar as strings em minúsculo para melhorar nossa análise de correlação

In [15]:
import json

def extrair_tipo_categoria(produto):
    # Substituir traços por espaço e dividir o nome do produto
    partes = produto.replace('-', ' ').split()
    
    # O primeiro elemento é considerado tipo, o restante é a categoria
    tipo = partes[0].lower()  # Convertendo para minúsculas para uniformidade
    categoria = ' '.join(partes[1:]).lower()  # Unindo o restante como categoria
    
    return tipo, categoria

def transformar_produtos(input_json, output_json):
    # Ler o JSON de entrada
    with open(input_json, 'r', encoding='utf-8') as file:
        dados = json.load(file)

    # Novo dicionário para armazenar as compras com produtos transformados
    compras_transformadas = []

    # Iterar sobre as compras e seus produtos
    for item in dados:
        produtos_transformados = []  # Lista para produtos da compra atual
        produtos = item['produtos']
        for produto in produtos:
            tipo, categoria = extrair_tipo_categoria(produto)
            produtos_transformados.append({
                "tipo": tipo,
                "categoria": categoria,
            })
        
        # Adicionar a compra com seus produtos transformados
        compras_transformadas.append({
            "produtos": produtos_transformados
        })

    # Salvar as compras transformadas em um novo JSON
    with open(output_json, 'w', encoding='utf-8') as file:
        json.dump(compras_transformadas, file, ensure_ascii=False, indent=4)

# Chamar a função
transformar_produtos('nova_saida.json', 'saida_transformada.json')


* Agora temos um arquivo json otimizado para a etapa de mineração de dados.

In [16]:
import pandas as pd

# Carregar o JSON
df = pd.read_json('saida_transformada.json')

# Explodir a coluna de produtos para facilitar a análise
produtos = df.explode('produtos')

# Criar um DataFrame a partir da lista de produtos
produtos_df = pd.json_normalize(produtos['produtos'])

# Análise de frequência
frequencia = produtos_df.value_counts().reset_index(name='count')

# Verificar as colunas antes de renomeá-las
# print(frequencia.head())  # Para entender a estrutura

# Renomear as colunas com base na estrutura que você tem
# O número de colunas aqui pode ser 3: tipo, categoria e count
if frequencia.shape[1] == 3:
    frequencia.columns = ['tipo', 'categoria', 'quantidade']
else:
    print("O número de colunas não corresponde ao esperado.")

# Exibir o resultado
print(frequencia)


        tipo          categoria  quantidade
0     queijo          mussarela          39
1        pao            frances          28
2     pastel             queijo          22
3   presunto              sadia          20
4       doce           goiabada          18
5       cafe             melita          17
6      refri          coca cola          17
7   presunto              seara          17
8     pastel             frango          17
9      refri              fanta          17
10  presunto           perdigao          16
11     refri              pepsi          16
12    pastel              carne          15
13      doce           amendoim          15
14       pao             gajeta          14
15      cafe             iguacu          13
16    queijo              minas          13
17      doce              leite          13
18       pao            cabrito          13
19      cafe             nestle          10
20    queijo             lanche           9
21    pastel  presunto e queijo 